Convert the DICOM Image into PNG


In [13]:
import os
import rich
import pydicom as dicom
import numpy as np
import pandas as pd 
from PIL import Image 

In [6]:
FILTERED_DATA_PATH = 'filtered_data/'
FILTERED_DATA_PNG_PATH = 'filtered_data_png/'

os.makedirs(FILTERED_DATA_PNG_PATH, exist_ok=True)

In [7]:
def convert_dicom_to_jpeg(name):
    image = dicom.dcmread (name)
    if 'PixelData' in image:
        image = image.pixel_array.astype(float)
        rescaled_image = (np.maximum(image, 0) / image.max()) * 255
        rescaled_image = np.uint8(rescaled_image)
        jpeg_image = Image.fromarray(rescaled_image)
        return jpeg_image 
    else:
        print(f"No PixelData found for {name})")
        return None

In [8]:
for name in os.listdir(FILTERED_DATA_PATH):
    full_path = os.path.join(FILTERED_DATA_PATH, name)
    jpeg_image = convert_dicom_to_jpeg(full_path)
    if jpeg_image:
        jpeg_image.save(FILTERED_DATA_PNG_PATH + name + '.png')

**Metadata extraction**

In [33]:
def list_all_metadata(dicom_file_path):
    dicom_data = dicom.dcmread(dicom_file_path)
    rich.print(dicom_data.dir())

In [ ]:
def list_all_metadata_with_values(dicom_file_path):
    dicom_data = dicom.dcmread(dicom_file_path)
    rich.print(dicom_data.dir)

In [ ]:
list_all_metadata(FILTERED_DATA_PATH + "I0000091")

Keep only relevant metadata 

In [31]:
def extract_metadata(dicom_file_path):
    dicom_data = dicom.dcmread(dicom_file_path)
    metadata = {
        'PatientID': dicom_data.PatientID if 'PatientID' in dicom_data else None,
        'PatientName': dicom_data.PatientName if 'PatientName' in dicom_data else None,
        'PatientAge': dicom_data.PatientAge if 'PatientAge' in dicom_data else None,
        'PatientSex': dicom_data.PatientSex if 'PatientSex' in dicom_data else None,
        'StudyDate': dicom_data.StudyDate if 'StudyDate' in dicom_data else None,
        "View Position": dicom_data.ViewPosition if 'ViewPosition' in dicom_data else None,
        "Animal Species": dicom_data.PatientSpeciesDescription if 'PatientSpeciesDescription' in dicom_data else None,
        "Weight": dicom_data.PatientWeight if 'PatientWeight' in dicom_data else None,
        "Type": dicom_data.PatientBreedDescription if 'PatientBreedDescription' in dicom_data else None,
    }
    return metadata


In [32]:
metadata_list = []
for filename in os.listdir(FILTERED_DATA_PATH):
    metadata = extract_metadata(FILTERED_DATA_PATH + filename)
    metadata_list.append(metadata)

metadata_df = pd.DataFrame(metadata_list)

In [35]:
metadata_df.to_csv('filtered_data/metadata.csv', index=False)